In [1]:
import os
%pwd

'/Users/swapneelpatil/Documents/Personal/chicken-disease-classification/research'

In [2]:
os.chdir("../")
%pwd

'/Users/swapneelpatil/Documents/Personal/chicken-disease-classification'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PreapareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    
    def get_prepare_callback_config(self) -> PreapareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = PreapareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= config.checkpoint_model_filepath
        )
        
        return prepare_callback_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [14]:
class PrepareCallbacks:
    def __init__(self, config: PreapareCallbacksConfig) :
        self.config = config
        
    @property
    def _create_tb_callbacks(self) :
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=str(tb_running_log_dir))
    
    
    @property
    def _create_ckpt_callbacks(self) :
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_ckpt_callbacks(self) :
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
        

In [15]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallbacks(config= prepare_callback_config)
    callback_list = prepare_callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-10-24 18:01:41,658: INFO: common: Successfully read config/config.yaml as <class 'dict'>]
[2023-10-24 18:01:41,659: INFO: common: Successfully read params.yaml as <class 'dict'>]


In [17]:
# 2 hr 6 min 